# Load mutual fund return data from csv

In [103]:
# Import all the packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import FuncFormatter
import sklearn.linear_model as lm
import statsmodels.api as sm
from tqdm.notebook import trange, tqdm

In [104]:
# Read data into dataframe
## Read the data with the 500 funds through time with only funds that contain management fees and turnover data
data = pd.read_csv('fund_data_largest_500_with_fees.csv')

# Convert the 'caldt' to a Python datetime and then to the Fama-French date format (YYYYMM)
data['Date'] = pd.to_datetime(data['caldt'], format="%Y%m%d")
data['YYYYMM'] = data['Date'].dt.strftime('%Y%m').astype(int)

# Set multi-column index (YYYYMM and crsp_fundno), but keep original columns
data.set_index(['YYYYMM','crsp_fundno'], drop=False, inplace=True)

# Print the top 3 rows
# print(data.head(3))
# print(data.columns)
# data.head(10).to_excel('Quick sample for inspection.xlsx')
# data.head()

# Load the Fama-French series

In [105]:
#  Load the 5 Factor into dataframe
famafrench = pd.read_csv('F-F_Research_Data_5_Factors_2x3.CSV', index_col=0, skiprows=3)

#  Load the 5 Factor into dataframe
momentum = pd.read_csv('F-F_Momentum_Factor.CSV', skiprows=13, index_col=0)

# Left-join the momentum dataframe to the famafrench dataframe
famafrench = famafrench.merge(momentum, left_index=True, right_index=True, how='left')

# Only select CAPM, Fama-French 3 models and 4-factors Carhart model
#famafrench = famafrench[['Mkt-RF', 'SMB', 'HML', 'RF', 'Mom']]

# Rename the momentum column to "WML"
famafrench.rename(columns={"Mom":"WML"}, inplace=True)

# Make returns into decimals instead of percentages
famafrench = famafrench / 100
display(famafrench)

,Mkt-RF,SMB,HML,RMW,CMA,RF,WML
196307,-0.0039,-0.0045,-0.0094,0.0066,-0.0115,0.0027,0.0100
196308,0.0507,-0.0082,0.0182,0.0040,-0.0040,0.0025,0.0103
196309,-0.0157,-0.0048,0.0017,-0.0076,0.0024,0.0027,0.0016
196310,0.0253,-0.0130,-0.0004,0.0275,-0.0224,0.0029,0.0314
196311,-0.0085,-0.0085,0.0170,-0.0045,0.0222,0.0027,-0.0075
...,...,...,...,...,...,...,...
202008,0.0763,-0.0094,-0.0294,0.0427,-0.0144,0.0001,0.0051
202009,-0.0363,0.0007,-0.0251,-0.0115,-0.0177,0.0001,0.0305
202010,-0.0210,0.0476,0.0403,-0.0060,-0.0053,0.0001,-0.0303
202011,0.1247,0.0675,0.0211,-0.0278,0.0105,0.0001,-0.1225


### Fama-French cumulative return plot (after 2000)

In [106]:
# Get sample period
famafrench_sample = famafrench[famafrench.index > 199912]

# Calculate cumulative returns on the dataframe
#famafrench_cumprod = np.cumprod(1 + famafrench_sample, axis=0) - 1   # Cumulative ARITHMETIC returns
famafrench_cumprod = np.cumsum(np.log(1 + famafrench_sample), axis=0)  # Cumulative LOG returns

# Add Python datetime to the dataframe (new column called "Date")
famafrench_cumprod['Date'] = pd.to_datetime(famafrench_sample.index, format="%Y%m")

# Plot the cumulative fama-french factors over time
# figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
# plt.plot(famafrench_cumprod['Date'], famafrench_cumprod[['Mkt-RF','SMB','HML','RF','WML','RMW','CMA']])
# plt.title('Cumulative log returns over time')
# plt.legend(['Market','Size','Value','Riskfree','Momentum', 'Profitability', 'Investments'])
# plt.xlabel('Time (months)')
# plt.ylabel('Log cumulative returns')
# plt.show()

# Adding new variables to the mutual funds based on rolling windows

### Calculating the 1-year alpha, market beta, HML beta, 12-month returns for each fund at each month

In [99]:
# Initialize list that is going to keep track of all our results
# I'm using a list here because that is the fastest object in Python!
# I also tried to directly save the results to the 'data' dataframe inside of the loops but that was 4x slower.
results = []

# Specify type of model
reg = lm.LinearRegression()

# Get the string representation of the months starting in 1992
dates = famafrench.index[famafrench.index >= 199200].unique()

# Starting index
lookback = 12            # Lookback period is 12 months (or 3 months)

# Loop over the dates
for period in trange(lookback, len(dates)):
    
    # Get the current date
    current_date = dates[period]
    
    # Get all dates of 36 months ago until last month
    lookback_period = dates[period-lookback:period]
    
    # Print some information
    print('Iteration {} at date {}, lookback period is {} to {} ({:.0f} months)'.format(
        period, current_date, lookback_period[0], lookback_period[-1], len(lookback_period)))

    # Get corresponding Fama-French data
    ff_history_index = famafrench.index.isin(lookback_period)
    ff_history = famafrench.loc[ff_history_index]
    
    # Get all funds at this period in time (in this iteration)
    funds_this_period_index = data.index.get_level_values('YYYYMM') == current_date
    funds_this_period = data.loc[funds_this_period_index, 'crsp_fundno'].unique()
    
    # Get all historical data for the funds that exist in this period in time
    funds_historical_data_index = np.logical_and(
        data.index.get_level_values('YYYYMM').isin(lookback_period),
        data.index.get_level_values('crsp_fundno').isin(funds_this_period)
    )
    funds_historical_data = data.loc[funds_historical_data_index]
    
    # Loop over all funds THIS PERIOD
    for fund in funds_this_period:
        
        # Get the row positions for the data for this fund   
        fund_data_index = funds_historical_data.index.get_level_values('crsp_fundno') == fund
        
        # Get the individual data for this fund
        # And make new index that corresponds to the ff_history index (this makes the merge below faster!)
        fund_data = funds_historical_data.loc[fund_data_index].set_index('YYYYMM')
        
        # If this specific fund has less than 25 observations, ignore it! For 1 month lookback, change this to 2
        if fund_data.shape[0] < 12:
            continue
        
        # Join the regression data together
        reg_data = ff_history.merge(fund_data['mret'], left_index=True, right_index=True, how='left')
        
        # Calculate the fund returns over the previous 12 months (or less)
        ret_12_months = np.product(1 + reg_data.tail(12)['mret']) - 1
        
        # Remove rows with no data
        reg_data = reg_data.dropna()
        
        # Calculate excess returns for the fund
        reg_data['Excess fund return'] = reg_data['mret'] - reg_data['RF']
            
        # Get the x-values for the regression
        x = reg_data[['Mkt-RF','HML']].values.reshape(reg_data.shape[0], 2)
        
        # Get the y-values for the regression
        y = reg_data['Excess fund return'].values.reshape(reg_data.shape[0], 1)
        
        # Fit regression
        reg.fit(x, y)
        
        # Append the calculations to our 'results' list
        # Note that the order in which we save the results matters! (see comment with [!!!] at the bottom of this code)
        
        #                 YYYYMM   crsp_fundno     Alpha        Market beta        HML beta        12M return
        #                    v          v            v               v                v                v
        #                    v          v            v               v                v                v
        results.append([current_date, fund, reg.intercept_[0], reg.coef_[0][0], reg.coef_[0][1], ret_12_months])

# This is the end of the loops

# We now turn our 'results' list into a dataframe (so we can merge it with the 'data' dataframe below)
# [!!!] Note that we need to specify the order of the results using (the correct) column names
results = pd.DataFrame(results, columns=['YYYYMM','crsp_fundno','Alpha','Market beta','HML beta','12M return'])

# Create multi-index (so it matches the 'data' dataframe index)
results.set_index(['YYYYMM','crsp_fundno'], inplace=True)

# Final step: merge our results back into the original 'data' dataframe. The matching is done based on the multi-index!
data = data.merge(results, left_index=True, right_index=True, how='left')

# DONE! 
# We just added new data to each fund at each moment in time based on historical (36-month) regressions!+b

print('Done.')

Iteration 12 at date 199301, lookback period is 199201 to 199212 (12 months)
Iteration 13 at date 199302, lookback period is 199202 to 199301 (12 months)
Iteration 14 at date 199303, lookback period is 199203 to 199302 (12 months)
Iteration 15 at date 199304, lookback period is 199204 to 199303 (12 months)
Iteration 16 at date 199305, lookback period is 199205 to 199304 (12 months)
Iteration 17 at date 199306, lookback period is 199206 to 199305 (12 months)
Iteration 18 at date 199307, lookback period is 199207 to 199306 (12 months)
Iteration 19 at date 199308, lookback period is 199208 to 199307 (12 months)
Iteration 20 at date 199309, lookback period is 199209 to 199308 (12 months)
Iteration 21 at date 199310, lookback period is 199210 to 199309 (12 months)
Iteration 22 at date 199311, lookback period is 199211 to 199310 (12 months)
Iteration 23 at date 199312, lookback period is 199212 to 199311 (12 months)
Iteration 24 at date 199401, lookback period is 199301 to 199312 (12 months)

Iteration 119 at date 200112, lookback period is 200012 to 200111 (12 months)
Iteration 120 at date 200201, lookback period is 200101 to 200112 (12 months)
Iteration 121 at date 200202, lookback period is 200102 to 200201 (12 months)
Iteration 122 at date 200203, lookback period is 200103 to 200202 (12 months)
Iteration 123 at date 200204, lookback period is 200104 to 200203 (12 months)
Iteration 124 at date 200205, lookback period is 200105 to 200204 (12 months)
Iteration 125 at date 200206, lookback period is 200106 to 200205 (12 months)
Iteration 126 at date 200207, lookback period is 200107 to 200206 (12 months)
Iteration 127 at date 200208, lookback period is 200108 to 200207 (12 months)
Iteration 128 at date 200209, lookback period is 200109 to 200208 (12 months)
Iteration 129 at date 200210, lookback period is 200110 to 200209 (12 months)
Iteration 130 at date 200211, lookback period is 200111 to 200210 (12 months)
Iteration 131 at date 200212, lookback period is 200112 to 20021

Iteration 225 at date 201010, lookback period is 200910 to 201009 (12 months)
Iteration 226 at date 201011, lookback period is 200911 to 201010 (12 months)
Iteration 227 at date 201012, lookback period is 200912 to 201011 (12 months)
Iteration 228 at date 201101, lookback period is 201001 to 201012 (12 months)
Iteration 229 at date 201102, lookback period is 201002 to 201101 (12 months)
Iteration 230 at date 201103, lookback period is 201003 to 201102 (12 months)
Iteration 231 at date 201104, lookback period is 201004 to 201103 (12 months)
Iteration 232 at date 201105, lookback period is 201005 to 201104 (12 months)
Iteration 233 at date 201106, lookback period is 201006 to 201105 (12 months)
Iteration 234 at date 201107, lookback period is 201007 to 201106 (12 months)
Iteration 235 at date 201108, lookback period is 201008 to 201107 (12 months)
Iteration 236 at date 201109, lookback period is 201009 to 201108 (12 months)
Iteration 237 at date 201110, lookback period is 201010 to 20110

Iteration 331 at date 201908, lookback period is 201808 to 201907 (12 months)
Iteration 332 at date 201909, lookback period is 201809 to 201908 (12 months)
Iteration 333 at date 201910, lookback period is 201810 to 201909 (12 months)
Iteration 334 at date 201911, lookback period is 201811 to 201910 (12 months)
Iteration 335 at date 201912, lookback period is 201812 to 201911 (12 months)
Iteration 336 at date 202001, lookback period is 201901 to 201912 (12 months)
Iteration 337 at date 202002, lookback period is 201902 to 202001 (12 months)
Iteration 338 at date 202003, lookback period is 201903 to 202002 (12 months)
Iteration 339 at date 202004, lookback period is 201904 to 202003 (12 months)
Iteration 340 at date 202005, lookback period is 201905 to 202004 (12 months)
Iteration 341 at date 202006, lookback period is 201906 to 202005 (12 months)
Iteration 342 at date 202007, lookback period is 201907 to 202006 (12 months)
Iteration 343 at date 202008, lookback period is 201908 to 20200

## Print the first 10.000 rows to Excel for manual inspection

In [12]:
# Print the first 10.000 rows to Excel for inspection:
#data.head(10000).to_excel('Fund data with new attributes - inspection.xlsx')

# Creating a strategy based on mutual funds ("rank portfolios")
### The idea is simple: once a year we select the mutual funds with the highest return over the previous year. We then buy those mutual funds (equally-weighted) and hold onto them for a year. We then "rebalance" by repeating the procedure (selecting the new best funds), hold onto them for a year, etc.

In [101]:
# Settings
lookforward = 12                # Evaluation period is 12 (or 3 or 1) months (one year into the future). 
rebalance = lookforward         # Rebalancing frequency is equal to lookforward period (every 12 months)
number_quantiles = 10           # Number of quantile portfolios to create. For this example we will create DECILES.
#rank_variable = '12M return'    # Characteristic on which funds will be sorted into quantiles
rank_variable = 'mgmt_fee'    # Characteristic on which funds will be sorted into quantiles

# Get the string representation of the months starting in December 1992 (first rebalancing moment)
dates = famafrench.index[famafrench.index >= 199212].unique()

# Initialize results
quantile_portfolios = pd.DataFrame(data=None, index=dates, columns=range(0, number_quantiles))
quantile_portfolios_other_variable_pd = pd.DataFrame(data=None, index=dates, columns=range(0, number_quantiles))

# Loop over the dates
for period in trange(0, len(dates), rebalance):
    
    #print(f'the period is {period}')

    # Get the current date
    current_date = dates[period]
    #print(f'the current date is {current_date}')
    
    # Get all dates of the next year (lookforward window)
    lookforward_period = dates[period+1:period+lookforward+1]
    #print(f'the lookforward period is {lookforward_period}')
    
    # Assign a cumulative number for each period in the future (so we can group by this later)
    quantile_portfolios_index = quantile_portfolios.index.isin(lookforward_period)
    quantile_portfolios.loc[quantile_portfolios_index, 'Holding period'] = range(1, len(lookforward_period)+1)
    
    # Get all funds at this period in time (in this iteration)
    funds_this_period_index = data.index.get_level_values('YYYYMM') == current_date
    funds_this_period = data.loc[funds_this_period_index]
    
    # If less than 100 funds in total, ignore the code below
    if funds_this_period[rank_variable].count() < 100:
        continue
    
    # Sort the funds into quantiles (using pandas 'qcut' function)
    quantiles_index = pd.qcut(funds_this_period[rank_variable], number_quantiles, labels=range(0, number_quantiles))
    
    # Loop over each quantile
    for q in range(0, number_quantiles):
        
        # Get the fund identifiers that are part of this quantile
        funds_in_quantile = quantiles_index.index.get_level_values('crsp_fundno')[quantiles_index == q]
        
        # Get the forward-looking returns of the funds that belong to this quantile
        fund_returns_index = np.logical_and(
            data.index.get_level_values('YYYYMM').isin(lookforward_period),
            data.index.get_level_values('crsp_fundno').isin(funds_in_quantile)
        )
        fund_returns = data.loc[fund_returns_index, 'mret']
        fund_other_rank_variable = data.loc[fund_returns_index, rank_variable]
        
        # Put the dates as columns (instead of having a multi-column index, make separate columns for each date)
        fund_returns = fund_returns.unstack(level='YYYYMM')
        fund_other_rank_variable = fund_other_rank_variable.unstack(level='YYYYMM')
        
        # Define the weighting scheme.
        # For now, we will created equally-weighted quantile portfolios.
        # We assume that if a mutual fund closes, we can reinvest our redeemed money in the remaining funds.
        # These weights are the basis for the other rank variable 
        fund_weights = fund_returns.notna().astype(int)
        fund_weights = fund_weights.div(fund_weights.sum(axis=0), axis=1)
        
        # Multiply returns with weights to get the return of the quantile portfolio and also for the other variable
        quantile_portfolio_return = (fund_returns * fund_weights).sum(axis=0, skipna=True)
        quantile_portfolios_other_variable = (fund_other_rank_variable * fund_weights).sum(axis=0, skipna=True)
        
        # Save the future returns for this quantile in the 'quantile_portfolios' dataframe
        quantile_portfolios_index = quantile_portfolios.index.isin(lookforward_period)
        quantile_portfolios.loc[quantile_portfolios_index, q] = quantile_portfolio_return
        quantile_portfolios_other_variable_index = quantile_portfolios_other_variable_pd.index.isin(lookforward_period)
        quantile_portfolios_other_variable_pd.loc[quantile_portfolios_other_variable_index, q] = quantile_portfolios_other_variable        
        
print('Quantile portfolios are created.')


Quantile portfolios are created.


### This is for creating a lag of 1 month. Run only if necessary

In [ ]:
# # Settings
# lag = 1                         # lag of 1 month
# lookforward = 12                # Evaluation period is 12 (or 3 or 1) months (one year into the future). 
# rebalance = lookforward + lag   # Rebalancing frequency is NOT equal to lookforward period (every 13 months)
# number_quantiles = 10           # Number of quantile portfolios to create. For this example we will create DECILES.
# #rank_variable = '12M return'   # Characteristic on which funds will be sorted into quantiles
# rank_variable = 'mgmt_fee'      # Characteristic on which funds will be sorted into quantiles

# # Get the string representation of the months starting in December 1992 (first rebalancing moment)
# dates = famafrench.index[famafrench.index >= 199212].unique()

# # Initialize results
# quantile_portfolios = pd.DataFrame(data=None, index=dates, columns=range(0, number_quantiles))
# quantile_portfolios_other_variable_pd = pd.DataFrame(data=None, index=dates, columns=range(0, number_quantiles))

# # Loop over the dates
# for period in trange(0, len(dates), rebalance):
    
#     #print(f'the period is {period}')

#     # Get the current date
#     current_date = dates[period]
#     #print(f'the current date is {current_date}')
    
#     # Get all dates of the next year (lookforward window)
#     lookforward_period = dates[period+1:period+lookforward+1]
#     #print(f'the lookforward period is {lookforward_period}')
    
#     # Assign a cumulative number for each period in the future (so we can group by this later)
#     quantile_portfolios_index = quantile_portfolios.index.isin(lookforward_period)
#     quantile_portfolios.loc[quantile_portfolios_index, 'Holding period'] = range(1, len(lookforward_period)+1)
    
#     # Get all funds at this period in time (in this iteration)
#     funds_this_period_index = data.index.get_level_values('YYYYMM') == current_date
#     funds_this_period = data.loc[funds_this_period_index]
    
#     # If less than 100 funds in total, ignore the code below
#     if funds_this_period[rank_variable].count() < 100:
#         continue
    
#     # Sort the funds into quantiles (using pandas 'qcut' function)
#     quantiles_index = pd.qcut(funds_this_period[rank_variable], number_quantiles, labels=range(0, number_quantiles))
    
#     # Loop over each quantile
#     for q in range(0, number_quantiles):
        
#         # Get the fund identifiers that are part of this quantile
#         funds_in_quantile = quantiles_index.index.get_level_values('crsp_fundno')[quantiles_index == q]
        
#         # Get the forward-looking returns of the funds that belong to this quantile
#         fund_returns_index = np.logical_and(
#             data.index.get_level_values('YYYYMM').isin(lookforward_period),
#             data.index.get_level_values('crsp_fundno').isin(funds_in_quantile)
#         )
#         fund_returns = data.loc[fund_returns_index, 'mret']
#         fund_other_rank_variable = data.loc[fund_returns_index, rank_variable]
        
#         # Put the dates as columns (instead of having a multi-column index, make separate columns for each date)
#         fund_returns = fund_returns.unstack(level='YYYYMM')
#         fund_other_rank_variable = fund_other_rank_variable.unstack(level='YYYYMM')
        
#         # Define the weighting scheme.
#         # For now, we will created equally-weighted quantile portfolios.
#         # We assume that if a mutual fund closes, we can reinvest our redeemed money in the remaining funds.
#         # These weights are the basis for the other rank variable 
#         fund_weights = fund_returns.notna().astype(int)
#         fund_weights = fund_weights.div(fund_weights.sum(axis=0), axis=1)
        
#         # Multiply returns with weights to get the return of the quantile portfolio and also for the other variable
#         quantile_portfolio_return = (fund_returns * fund_weights).sum(axis=0, skipna=True)
#         quantile_portfolios_other_variable = (fund_other_rank_variable * fund_weights).sum(axis=0, skipna=True)
        
#         # Save the future returns for this quantile in the 'quantile_portfolios' dataframe
#         quantile_portfolios_index = quantile_portfolios.index.isin(lookforward_period)
#         quantile_portfolios.loc[quantile_portfolios_index, q] = quantile_portfolio_return
#         quantile_portfolios_other_variable_index = quantile_portfolios_pd.index.isin(lookforward_period)
#         quantile_portfolios_other_variable_pd.loc[quantile_portfolios_other_variable_index, q] = quantile_portfolios_other_variable        
        
# print('Quantile portfolios are created.')

### Plot cumulative returns of quantile portfolios

In [ ]:
# # Calculate cumulative log returns of quantile portfolios
# quantile_portfolios_cumprod = np.cumprod(1 + quantile_portfolios, axis=0) - 1

# # Add Python datetime to the dataframe (new column called "Date")
# quantile_portfolios_cumprod['Date'] = pd.to_datetime(quantile_portfolios_cumprod.index, format="%Y%m")

# # Plot the cumulative fama-french factors over time
# figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
# plt.plot(quantile_portfolios_cumprod['Date'], quantile_portfolios_cumprod[range(0,number_quantiles)])
# plt.title('Cumulative returns over time')
# plt.legend(["Quantile {}".format(q) for q in range(1,number_quantiles+1)])
# plt.xlabel('Time (months)')
# plt.ylabel('Cumulative arithmetic returns')
# plt.show()

### Plot holding-period return for each quantile
_Edit: this plot now works for any holding period_

In [ ]:
# # Get the string representation of the months starting in December 1992 (first rebalancing moment)
# dates = famafrench.index[famafrench.index >= 199212].unique()

# # Because portfolios are rebalanced every December, we can simply keep track of the return in each month
# portfolio_returns = quantile_portfolios.copy()

# # Calculate the average returns per portfolio, per month
# returns_per_month = portfolio_returns.fillna(0).groupby(['Holding period']).mean()

# # Calculate the holding-period cumulative return
# holding_returns = np.cumprod(1 + returns_per_month, axis=0) - 1

# # Add a column with the average return per holding period
# holding_returns['Average'] = holding_returns.mean(axis=1)

# # For each fund, subtract the average return
# for q in range(0,number_quantiles):
#     holding_returns[q] = holding_returns[q] - holding_returns['Average']
    
# # Plot the cumulative fama-french factors over time
# figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
# plt.plot(holding_returns.index, holding_returns[range(0, number_quantiles)])
# plt.title('Cumulative returns over holding period')
# plt.legend(["Q{}".format(q) for q in range(1, number_quantiles+1)], loc='lower left')
# plt.xlabel('Holding period (months)')
# plt.ylabel('Cumulative returns in excess of the average')
# plt.show()

### Quantile portfolio statistics - Multi Factor

In [178]:
# Multi-factor
factor_model = 'Multi-Factor' 

# Specify portfolios to analyze
portfolios = quantile_portfolios

# Load Fama-French factors
ff = famafrench[famafrench.index.isin(portfolios.index)]

# Merge together
portfolios = portfolios.merge(ff, left_index=True, right_index=True, how='left')

# Initialize results
cols = ["Management fees", "Avg returns", "Avg excess returns", "Volatility", "Sharpe ratio", 
        "Alpha", "Std. error (α)", "t-stat (α)", "p-value (α)", "Beta",
        "SMB", "HML", "WML", "RMW", 'CMA', "R2 adjusted"]
results = pd.DataFrame(data=None, index=portfolios.columns, columns=cols)

# Calculate management fees
results["Management fees"] = quantile_portfolios_other_variable_pd.mean() * 12

# Calculate average returns
results["Avg returns"] = portfolios.mean() * 12
results.loc[:, "Avg excess returns"] = results.loc[:, "Avg returns"] - results.loc["RF", "Avg returns"]

# Calculate volatility of returns
results["Volatility"] = portfolios.std() * np.sqrt(12)

# Calculate Sharpe ratio
results["Sharpe ratio"] = results["Avg excess returns"] / results["Volatility"]

# Loop over each portfolio
for quantile in range(0, number_quantiles):
    
    # Find index of available data
    index = portfolios[quantile].notna()
    
    # Get x and y variables
    x = portfolios.loc[index, ['Mkt-RF', 'SMB', 'HML', 'WML', 'RMW', 'CMA']].astype(float)
    y = portfolios.loc[index, quantile].astype(float) - portfolios.loc[index, 'RF']
    if quantile == 0:
        matrix_factor_returns = y[0:251]
    else:
        matrix_factor_returns = np.vstack((matrix_res,y[0:251]))
    
    # Note! Need to add intercept myself using this syntax
    x = sm.add_constant(x)
    
    # Fit regression
    model = sm.OLS(y, x).fit()   # Note! y and x are passed to this function in reverse order. It differs per package!
    
    # Save alpha
    results.loc[quantile, "Alpha"] = model.params[0] * 12
    
    # Save standard error of the alpha
    results.loc[quantile, "Std. error (α)"] = model.bse[0]
    
    # Save tstat of the alpha
    results.loc[quantile, "t-stat (α)"] = model.tvalues[0]
    
    # Save p-value of the alpha
    results.loc[quantile, "p-value (α)"] = model.pvalues[0]
    
    # Save beta
    results.loc[quantile, "Beta"] = model.params[1]
    
    # Save SMB
    results.loc[quantile, "SMB"] = model.params[2]
    
    # Save HML
    results.loc[quantile, "HML"] = model.params[3]
    
    # Save RF
    #results.loc[quantile, "RF"] = model.params[4]
    
    # Save WML
    results.loc[quantile, "WML"] = model.params[4]
    
     # Save RMW
    results.loc[quantile, "RMW"] = model.params[5]
    
     # Save CMA
    results.loc[quantile, "CMA"] = model.params[6]
    
    # Save R squared adjusted
    results.loc[quantile, "R2 adjusted"] = model.rsquared_adj
    
    # Save residuals
    vector_resid = model.resid.values[0:251]
    if quantile == 0:
        matrix_res = vector_resid
    else:
        matrix_res = np.vstack((matrix_res,vector_resid))
    
# Only keep results for relevant factors
results = results.loc[range(0, number_quantiles)]
results.index = ["Q{}".format(q) for q in range(1, number_quantiles+1)]

# Print final results
print(f'Based on {factor_model} model, \n\
{lookback} month formation period and {lookforward} month holding period,')
if lookforward != rebalance:
    print(f'with a lag of {rebalance-lookforward} month,')
print('the final results are (all numbers annualized):')
display(results.T)

Based on Multi-Factor model, 
12 month formation period and 12 month holding period,
the final results are (all numbers annualized):


,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
Management fees,0.858648,2.01374,3.25046,4.45188,5.64816,6.61188,7.33924,8.1159,9.37922,11.8717
Avg returns,0.0780334,0.0866592,0.0744563,0.0742097,0.0628151,0.0727015,0.0721068,0.0877381,0.071618,0.0653447
Avg excess returns,0.054977,0.0636028,0.0513999,0.0511533,0.0397587,0.0496451,0.0490504,0.0646817,0.0485617,0.0422883
Volatility,0.15501,0.155485,0.154041,0.156863,0.164117,0.15302,0.156597,0.168363,0.16926,0.16773
Sharpe ratio,0.354668,0.409062,0.333677,0.326101,0.242258,0.324436,0.313228,0.384179,0.286906,0.252121
Alpha,-0.00379367,0.00164921,-0.00169634,-0.00327665,-0.00870896,-0.00467677,0.00191471,0.0101277,-0.00759291,-0.00843069
Std. error (α),0.000295111,0.000644409,0.000531274,0.000454916,0.000434431,0.000634062,0.000523942,0.000857149,0.000516987,0.000667786
t-stat (α),-1.07125,0.213271,-0.26608,-0.600231,-1.67057,-0.614657,0.304536,0.984633,-1.2239,-1.05207
p-value (α),0.285114,0.831293,0.790401,0.548907,0.096084,0.539352,0.760978,0.325776,0.222165,0.293804
Beta,0.988826,0.967158,0.951315,0.95383,0.969162,0.931178,0.930739,0.997874,1.00016,0.915814


In [177]:
matrix_y

array([[ 0.00237126, -0.00861378,  0.01179809, ...,  0.0046263 ,
         0.00224411, -0.00207681],
       [ 0.01117314,  0.00373347,  0.00759908, ...,  0.00364815,
        -0.00632699, -0.00951852],
       [ 0.00602508,  0.02086251,  0.00554359, ...,  0.02412807,
         0.01177576, -0.06826424],
       ...,
       [-0.00124853,  0.02358279, -0.00056648, ...,  0.00377886,
         0.00843241, -0.00211683],
       [ 0.00255942,  0.02349367,  0.01417387, ...,  0.00859823,
         0.00203879, -0.02225672],
       [-0.02832206,  0.16821239, -0.00746182, ..., -0.03531658,
        -0.00504587,  0.13025721]])

### Quantile portfolio statistics - Fama French 5 factor

In [ ]:
# Fama French 5 factor
factor_model = 'Fama-French 5 factor'

# Specify portfolios to analyze
portfolios = quantile_portfolios

# Load Fama-French factors
ff = famafrench[famafrench.index.isin(portfolios.index)]

# Merge together
portfolios = portfolios.merge(ff, left_index=True, right_index=True, how='left')

# Initialize results
cols = ["Avg returns", "Avg excess returns", "Volatility", "Sharpe ratio", 
        "Alpha", "Std. error (α)", "t-stat (α)", "p-value (α)", "Beta",
        "SMB", "HML", "RMW", 'CMA', "R2 adjusted"]
results = pd.DataFrame(data=None, index=portfolios.columns, columns=cols)

# Calculate average returns
results["Avg returns"] = portfolios.mean() * 12
results.loc[:, "Avg excess returns"] = results.loc[:, "Avg returns"] - results.loc["RF", "Avg returns"]

# Calculate volatility of returns
results["Volatility"] = portfolios.std() * np.sqrt(12)

# Calculate Sharpe ratio
results["Sharpe ratio"] = results["Avg excess returns"] / results["Volatility"]

# Initialize matrix of model residuals and factor returns
matrix_res = []
matrix_y = []

# Loop over each portfolio
for quantile in range(0, number_quantiles):
    
    # Find index of available data
    index = portfolios[quantile].notna()
    
    # Get x and y variables
    x = portfolios.loc[index, ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']].astype(float)
    y = portfolios.loc[index, quantile].astype(float) - portfolios.loc[index, 'RF']
    matrix_y.append(y.values)
    matrix_y_t = np.transpose(np.array(matrix_y))
    
    # Note! Need to add intercept myself using this syntax
    x = sm.add_constant(x)
    
    # Fit regression
    model = sm.OLS(y, x).fit()   # Note! y and x are passed to this function in reverse order. It differs per package!
    
    # Save alpha
    results.loc[quantile, "Alpha"] = model.params[0] * 12
    
    # Save standard error of the alpha
    results.loc[quantile, "Std. error (α)"] = model.bse[0]
    
    # Save tstat of the alpha
    results.loc[quantile, "t-stat (α)"] = model.tvalues[0]
    
    # Save p-value of the alpha
    results.loc[quantile, "p-value (α)"] = model.pvalues[0]
    
    # Save beta
    results.loc[quantile, "Beta"] = model.params[1]
    
    # Save SMB
    results.loc[quantile, "SMB"] = model.params[2]
    
    # Save HML
    results.loc[quantile, "HML"] = model.params[3]
    
    # Save RF
    #results.loc[quantile, "RF"] = model.params[4]
    
     # Save RMW
    results.loc[quantile, "RMW"] = model.params[4]
    
     # Save CMA
    results.loc[quantile, "CMA"] = model.params[5]
    
    # Save R squared adjusted
    results.loc[quantile, "R2 adjusted"] = model.rsquared_adj
    
    # Save residuals
    matrix_res.append(model.resid.values)
    matrix_res_t = np.transpose(np.array(matrix_res))
    
    
# Only keep results for relevant factors
results = results.loc[range(0, number_quantiles)]
results.index = ["Q{}".format(q) for q in range(1, number_quantiles+1)]

# Print final results
print(f'Based on {factor_model} model, \n\
{lookback} month formation period and {lookforward} month holding period,')
if lookforward != rebalance:
    print(f'with a lag of {rebalance-lookforward} month,')
print('the final results are (all numbers annualized):')
display(results.T)

### Quantile portfolio statistics - Carhart 4 factor

In [ ]:
# Carhart
factor_model = 'Carthart 4-factor'

# Specify portfolios to analyze
portfolios = quantile_portfolios

# Load Fama-French factors
ff = famafrench[famafrench.index.isin(portfolios.index)]

# Merge together
portfolios = portfolios.merge(ff, left_index=True, right_index=True, how='left')

# Initialize results
cols = ["Avg returns", "Avg excess returns", "Volatility", "Sharpe ratio", 
        "Alpha", "Std. error (α)", "t-stat (α)", "p-value (α)", "Beta",
        "SMB", "HML", "WML", "R2 adjusted"]
results = pd.DataFrame(data=None, index=portfolios.columns, columns=cols)

# Calculate average returns
results["Avg returns"] = portfolios.mean() * 12
results.loc[:, "Avg excess returns"] = results.loc[:, "Avg returns"] - results.loc["RF", "Avg returns"]

# Calculate volatility of returns
results["Volatility"] = portfolios.std() * np.sqrt(12)

# Calculate Sharpe ratio
results["Sharpe ratio"] = results["Avg excess returns"] / results["Volatility"]

# Initialize matrix of model residuals and factor returns
matrix_res = []
matrix_y = []

# Loop over each portfolio
for quantile in range(0, number_quantiles):
    
    # Find index of available data
    index = portfolios[quantile].notna()
    
    # Get x and y variables
    x = portfolios.loc[index, ['Mkt-RF', 'SMB', 'HML', 'WML']].astype(float)
    y = portfolios.loc[index, quantile].astype(float) - portfolios.loc[index, 'RF']
    matrix_y.append(y.values)
    matrix_y_t = np.transpose(np.array(matrix_y))
    
    # Note! Need to add intercept myself using this syntax
    x = sm.add_constant(x)
    
    # Fit regression
    model = sm.OLS(y, x).fit()   # Note! y and x are passed to this function in reverse order. It differs per package!
    
    # Save alpha
    results.loc[quantile, "Alpha"] = model.params[0] * 12
    
    # Save standard error of the alpha
    results.loc[quantile, "Std. error (α)"] = model.bse[0]
    
    # Save tstat of the alpha
    results.loc[quantile, "t-stat (α)"] = model.tvalues[0]
    
    # Save p-value of the alpha
    results.loc[quantile, "p-value (α)"] = model.pvalues[0]
    
    # Save beta
    results.loc[quantile, "Beta"] = model.params[1]
    
    # Save SMB
    results.loc[quantile, "SMB"] = model.params[2]
    
    # Save HML
    results.loc[quantile, "HML"] = model.params[3]
    
    # Save RF
    #results.loc[quantile, "RF"] = model.params[4]
    
    # Save RML
    results.loc[quantile, "WML"] = model.params[4]
    
    # Save R squared adjusted
    results.loc[quantile, "R2 adjusted"] = model.rsquared_adj
    
    # Save residuals
    matrix_res.append(model.resid.values)
    matrix_res_t = np.transpose(np.array(matrix_res))
    
    
# Only keep results for relevant factors
results = results.loc[range(0, number_quantiles)]
results.index = ["Q{}".format(q) for q in range(1, number_quantiles+1)]

# Print final results
print(f'Based on {factor_model} model, \n\
{lookback} month formation period and {lookforward} month holding period,')
if lookforward != rebalance:
    print(f'with a lag of {rebalance-lookforward} month,')
print('the final results are (all numbers annualized):')
display(results.T)

### Quantile portfolio statistics - Fama French 3 factor

In [ ]:
# Fama French
factor_model = 'Fama-French 3 factor'

# Specify portfolios to analyze
portfolios = quantile_portfolios

# Load Fama-French factors
ff = famafrench[famafrench.index.isin(portfolios.index)]

# Merge together
portfolios = portfolios.merge(ff, left_index=True, right_index=True, how='left')

# Initialize results
cols = ["Avg returns", "Avg excess returns", "Volatility", "Sharpe ratio", 
        "Alpha", "Std. error (α)", "t-stat (α)", "p-value (α)", "Beta",
        "SMB", "HML"]#, "RF", "WML", "R2 adjusted"]
results = pd.DataFrame(data=None, index=portfolios.columns, columns=cols)

# Calculate average returns
results["Avg returns"] = portfolios.mean() * 12
results.loc[:, "Avg excess returns"] = results.loc[:, "Avg returns"] - results.loc["RF", "Avg returns"]

# Calculate volatility of returns
results["Volatility"] = portfolios.std() * np.sqrt(12)

# Calculate Sharpe ratio
results["Sharpe ratio"] = results["Avg excess returns"] / results["Volatility"]

# Initialize matrix of model residuals and factor returns
matrix_res = []
matrix_y = []

# Loop over each portfolio
for quantile in range(0, number_quantiles):
    
    # Find index of available data
    index = portfolios[quantile].notna()
    
    # Get x and y variables
    x = portfolios.loc[index, ['Mkt-RF', 'SMB', 'HML']].astype(float)
    y = portfolios.loc[index, quantile].astype(float) - portfolios.loc[index, 'RF']
    matrix_y.append(y.values)
    matrix_y_t = np.transpose(np.array(matrix_y))
    
    # Note! Need to add intercept myself using this syntax
    x = sm.add_constant(x)
    
    # Fit regression
    model = sm.OLS(y, x).fit()   # Note! y and x are passed to this function in reverse order. It differs per package!
    
    # Save alpha
    results.loc[quantile, "Alpha"] = model.params[0] * 12
    
    # Save standard error of the alpha
    results.loc[quantile, "Std. error (α)"] = model.bse[0]
    
    # Save tstat of the alpha
    results.loc[quantile, "t-stat (α)"] = model.tvalues[0]
    
    # Save p-value of the alpha
    results.loc[quantile, "p-value (α)"] = model.pvalues[0]
    
    # Save beta
    results.loc[quantile, "Beta"] = model.params[1]
    
    # Save SMB
    results.loc[quantile, "SMB"] = model.params[2]
    
    # Save HML
    results.loc[quantile, "HML"] = model.params[3]
    
    # Save RF
    #results.loc[quantile, "RF"] = model.params[4]
    
    # Save RML
    #results.loc[quantile, "WML"] = model.params[5]
    
    # Save R squared adjusted
    results.loc[quantile, "R2 adjusted"] = model.rsquared_adj
    
    # Save residuals
    matrix_res.append(model.resid.values)
    matrix_res_t = np.transpose(np.array(matrix_res))
    
    
# Only keep results for relevant factors
results = results.loc[range(0, number_quantiles)]
results.index = ["Q{}".format(q) for q in range(1, number_quantiles+1)]

# Print final results
print(f'Based on {factor_model} model, \n\
{lookback} month formation period and {lookforward} month holding period,')
if lookforward != rebalance:
    print(f'with a lag of {rebalance-lookforward} month,')
print('the final results are (all numbers annualized):')
display(results.T)

### Quantile portfolio statistics - CAPM

In [ ]:
# CAPM
factor_model = 'CAPM'

# Specify portfolios to analyze
portfolios = quantile_portfolios

# Load Fama-French factors
ff = famafrench[famafrench.index.isin(portfolios.index)]

# Merge together
portfolios = portfolios.merge(ff, left_index=True, right_index=True, how='left')

# Initialize results
cols = ["Avg returns", "Avg excess returns", "Volatility", "Sharpe ratio", 
        "Alpha", "Std. error (α)", "t-stat (α)", "p-value (α)", "Beta"]
        #"SMB", "HML", "RF", "WML", "R2 adjusted"]
results = pd.DataFrame(data=None, index=portfolios.columns, columns=cols)

# Calculate average returns
results["Avg returns"] = portfolios.mean() * 12
results.loc[:, "Avg excess returns"] = results.loc[:, "Avg returns"] - results.loc["RF", "Avg returns"]

# Calculate volatility of returns
results["Volatility"] = portfolios.std() * np.sqrt(12)

# Calculate Sharpe ratio
results["Sharpe ratio"] = results["Avg excess returns"] / results["Volatility"]

# Initialize matrix of model residuals and factor returns
matrix_res = []
matrix_y = []

# Loop over each portfolio
for quantile in range(0, number_quantiles):
    
    # Find index of available data
    index = portfolios[quantile].notna()
    
    # Get x and y variables
    x = portfolios.loc[index, ['Mkt-RF']].astype(float)
    y = portfolios.loc[index, quantile].astype(float) - portfolios.loc[index, 'RF']
    matrix_y.append(y.values)
    matrix_y_t = np.transpose(np.array(matrix_y))
    
    # Note! Need to add intercept myself using this syntax
    x = sm.add_constant(x)
    
    # Fit regression
    model = sm.OLS(y, x).fit()   # Note! y and x are passed to this function in reverse order. It differs per package!
    
    # Save alpha
    results.loc[quantile, "Alpha"] = model.params[0] * 12
    
    # Save standard error of the alpha
    results.loc[quantile, "Std. error (α)"] = model.bse[0]
    
    # Save tstat of the alpha
    results.loc[quantile, "t-stat (α)"] = model.tvalues[0]
    
    # Save p-value of the alpha
    results.loc[quantile, "p-value (α)"] = model.pvalues[0]
    
    # Save beta
    results.loc[quantile, "Beta"] = model.params[1]
    
    # Save SMB
    #results.loc[quantile, "SMB"] = model.params[2]
    
    # Save HML
    #results.loc[quantile, "HML"] = model.params[3]
    
    # Save RF
    #results.loc[quantile, "RF"] = model.params[4]
    
    # Save RML
    #results.loc[quantile, "WML"] = model.params[5]
    
    # Save R squared adjusted
    results.loc[quantile, "R2 adjusted"] = model.rsquared_adj
    
    # Save residuals
    matrix_res.append(model.resid.values)
    matrix_res_t = np.transpose(np.array(matrix_res))
    
    
# Only keep results for relevant factors
results = results.loc[range(0, number_quantiles)]
results.index = ["Q{}".format(q) for q in range(1, number_quantiles+1)]

# Print final results
print(f'Based on {factor_model} model, \n\
{lookback} month formation period and {lookforward} month holding period,')
if lookforward != rebalance:
    print(f'with a lag of {rebalance-lookforward} month,')
print('the final results are (all numbers annualized):')
display(results.T)

---

# To do, for advanced econometrics

In [107]:
# GRS results 
from numpy.linalg import inv
from scipy.stats import f

def GRS(alpha, resids, mu):
    # GRS test statistic
    # N assets, L factors, and T time points
    # alpha is a Nx1 vector of intercepts of the time-series regressions,
    # resids is a TxN matrix of residuals,
    # mu is a TxL matrix of factor returns --> the average returns 
    T, N = resids.shape
    L = mu.shape[1]
    mu_mean = np.nanmean(mu, axis=0)
    cov_resids = np.matmul(resids.T, resids) / (T-L-1)
    cov_fac = np.matmul(np.array(mu - np.nanmean(mu, axis=0)).T, np.array(mu - np.nanmean(mu, axis=0))) / T-1
    GRS = (T / N) * ((T - N - L) / (T - L - 1)) * ((np.matmul(np.matmul(alpha.T, inv(cov_resids)), alpha)) / (1 + (np.matmul(np.matmul(mu_mean.T, inv(cov_fac)), mu_mean))))
    pVal = 1-f.cdf(GRS, N, T - N - L)
    return GRS, pVal

In [179]:
GRS(results['Alpha'].values, matrix_res, matrix_factor_returns)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 251 is different from 10)

In [214]:
alpha, resids, mu = results['Alpha'].values, matrix_res, matrix_factor_returns

T, N = resids.shape
L = mu.shape[1]
mu_mean = np.nanmean(mu, axis=0)
cov_resids = np.matmul(resids.T, resids) / (T-L-1)
cov_fac = np.matmul(np.array(mu - np.nanmean(mu, axis=0)).T, np.array(mu - np.nanmean(mu, axis=0))) / T-1
GRS_1 = (T / N) * ((T - N - L) / (T - L - 1))  
#GRS_2 = ((np.matmul(np.matmul(alpha.T, inv(cov_resids)), alpha)))


resids.shape

(10, 251)

# That's it. Now it's up to you to study the code and practice with it!

In [ ]:
"""
  ____        _   _                   _                                                    
 |  _ \ _   _| |_| |__   ___  _ __   (_)___    __ ___      _____  ___  ___  _ __ ___   ___ 
 | |_) | | | | __| '_ \ / _ \| '_ \  | / __|  / _` \ \ /\ / / _ \/ __|/ _ \| '_ ` _ \ / _ \
 |  __/| |_| | |_| | | | (_) | | | | | \__ \ | (_| |\ V  V /  __/\__ \ (_) | | | | | |  __/
 |_|    \__, |\__|_| |_|\___/|_| |_| |_|___/  \__,_| \_/\_/ \___||___/\___/|_| |_| |_|\___|
        |___/                                                                              
        
"""
print('End of notebook.')

---